# Clarity - Interactive Tour

*Clarity is a **decidable** smart contract language that optimizes for predictability and security, designed by Blockstack. Smart contracts allow developers to encode essential business logic on a blockchain.*

*This notebook is part of a suite of noteboooks, designed for helping developpers getting started with Clarity.*


***
**This notebook was designed for encouraging experimentation. Feel free to execute the snippets, tweak values, refactor with your own Clarity code.**
***


# Chapter 3 - Data storage

In this chapter, we will be introducing two primitives for storing, updating and retrieving stateful data in Clarity programs, along with their associated functions.

## 1 - (define-data-var var-name type value)

`define-data-var` is used to define a new persisted variable for use in a smart contract. Such
variable are only modifiable by the current smart contract.

Persisted variable are defined with a type and a value.

Like other kinds of definition statements, `define-data-var` may only be used at the top level of a smart contract
definition (i.e., you cannot put a define statement in the middle of a function body).


In [ ]:
(define-data-var my-var int 0)

(print my-var) ;; returns an error!

When executing the snippet above, you will get an error. If you want to manipulate the value behind `my-var`, you need to explicitely do so.

Explicit read / write operations can be performed with the functions `var-get` and `var-set`.

### (var-set var-name expr1)
The `var-set` function sets the value associated with the input variable to the
inputted value.

### (var-get var-name)
The `var-set` function sets the value associated with the input variable to the
inputted value.

In [ ]:
(define-data-var my-var int 1)

(begin
    (var-set my-var 2)
    (var-get my-var))

***
## 2 - (define-map map-name ((key-name-0 key-type-0) ...) ((val-name-0 val-type-0) ...))
`define-map` is used to define a new datamap for use in a smart contract. Such
maps are only modifiable by the current smart contract.

Maps are defined with a key tuple type and value tuple type. These are defined using a list
of name and type pairs, e.g., a key type might be `((id int))`, which is a tuple with a single \"id\"
field of type `int`.

Like other kinds of definition statements, `define-map` may only be used at the top level of a smart contract
definition (i.e., you cannot put a define statement in the middle of a function body)

In [ ]:
(define-map kv-store ((key (string-ascii 8))) ((value (string-ascii 32))))

Clarity's language reference includes the following methods for manipulating this stateful data structures.



### (map-insert map-name key-tuple value-tuple)

The `map-insert` function sets the value associated with the input key to the
inputted value if and only if there is not already a value associated with the key in the map.
If an insert occurs, the function returns `true`. If a value already existed for
this key in the data map, the function returns `false`.

Note: the `value-tuple` requires 1 additional byte for storage in the materialized blockchain state,
and therefore the maximum size of a value that may be inserted into a map is MAX_CLARITY_VALUE - 1.


In [ ]:
(define-map users ((id int)) ((first-name (string-ascii 16)) (last-name (string-ascii 16))))

(begin
    ;; Insert once
    (map-insert users 
        { id: 1 } 
        { first-name: "satoshi", last-name: "nakamoto" }) ;; Returns true
    ;; Insert twice
    (map-insert users 
        { id: 1 }
        { first-name: "satoshi", last-name: "nakamoto" })) ;; Returns false

***
### (map-get? map-name key-tuple)
The `map-get?` function looks up and returns an entry from a contract's data map.
The value is looked up using `key-tuple`.
If there is no value associated with that key in the data map, the function returns a `none` option. Otherwise,
it returns `(some value)`.

In [ ]:
(define-map users ((id int)) ((first-name (string-ascii 16)) (last-name (string-ascii 16))))

(begin
    ;; Insert
    (map-insert users 
        { id: 1 } 
        { first-name: "satoshi", last-name: "nakamoto" }) ;; Returns true
    ;; Retrieve
    (print (list
        (map-get? users { id: 1 })
        (map-get? users { id: 2 }))))

***
### (map-set map-name key-tuple value-tuple)
The `map-set` function sets the value associated with the input key to the
inputted value. This function performs a _blind_ update; whether or not a value is already associated
with the key, the function overwrites that existing association.

Note: the `value-tuple` requires 1 additional byte for storage in the materialized blockchain state,
and therefore the maximum size of a value that may be inserted into a map is MAX_CLARITY_VALUE - 1.

In [ ]:
(define-map users ((id int)) ((first-name (string-ascii 16)) (last-name (string-ascii 16))))

(begin
    ;; Insert
    (map-set users 
        { id: 1 } 
        { first-name: "satoshi", last-name: "nakamoto" }) ;; Returns true
    ;; Retrieve
    (print (list
        (map-get? users { id: 1 })
        (map-get? users { id: 2 }))))

***
### (map-delete map-name key-tuple)
The `map-delete` function removes the value associated with the input key for
the given map. If an item exists and is removed, the function returns `true`.
If a value did not exist for this key in the data map, the function returns `false`.

In [ ]:
(define-map users ((id int)) ((first-name (string-ascii 16)) (last-name (string-ascii 16))))

(begin
    ;; Insert
    (map-set users 
        { id: 1 } 
        { first-name: "satoshi", last-name: "nakamoto" }) ;; Returns true
    ;; Abort execution if unable to retrieve Satoshi
    (try! (map-get? users { id: 1 }))
    (map-delete users { id: 1 })
    (print (map-get? users { id: 1 })))